# Laboratorio 6 - Análisis de Redes Sociales

#### Diego García 22404
#### Andrés Ortega 22305
#### Esteban Zambrano 22119

**Link del repositorio:**

https://github.com/DiegoGarV/lab6-DS

*PROBLEMA 2 - BERNARDO ARÉVALO*

## Limpieza de datos

In [ ]:
import os, json, re
from typing import Any, Dict, List
import pandas as pd

# Carpeta de datos / salida
os.makedirs("data", exist_ok=True)
INPUT_TXT = "../data/tioberny.txt" 

OUTPUT_CSV = "../data/converted_data.csv"
print("Leyendo de:", INPUT_TXT)


Leyendo de: ../data/tioberny.txt


Los json con los datos incluyen una gran cantidad de metadata, de la cuál no toda es necesaria. Tomando en cuenta los datos que se iban a analizar, se dicidió solo tomar en cuenta el username, la información del tweet, los hashtags y las menciones. Además, se tomó la misma información de los "quoted tweets" y se guardó como un tweet diferente. En el tweet donde se citó al otro tweet se incluyó en una columna llamada "fromUser" el username del usuario del tweet citado. Con esto se obtendrá el csv inicial para el laboratorio.

In [4]:
from typing import Any, Dict, List

# Detecta codificación
def sniff_encoding(path: str) -> str:
    with open(path, "rb") as f:
        raw = f.read(4)
    if raw.startswith(b"\xff\xfe\x00\x00") or raw.startswith(b"\x00\x00\xfe\xff"):
        return "utf-32"
    if raw.startswith(b"\xff\xfe") or raw.startswith(b"\xfe\xff"):
        return "utf-16"
    if raw.startswith(b"\xef\xbb\xbf"):
        return "utf-8-sig"
    return "utf-8"


"""
1) Detecta codificación.
2) Intenta parsear TODO como JSON (lista u objeto).
3) Si falla, interpreta como JSONL (una línea = un objeto JSON).
Ignora líneas vacías o malformadas en modo JSONL.
"""
def load_tweets_from_txt(path: str) -> List[Dict[str, Any]]:
    enc = sniff_encoding(path)
    print(f"[load] Codificación detectada: {enc}")

    # Caso 1: archivo entero como JSON
    with open(path, "r", encoding=enc) as f:
        content = f.read().strip()

    try:
        parsed = json.loads(content)
        if isinstance(parsed, list):
            return parsed
        elif isinstance(parsed, dict):
            return [parsed]
    except Exception:
        pass  # seguimos con JSONL

    # Caso 2: JSON Lines (una línea por objeto)
    tweets: List[Dict[str, Any]] = []
    bad, total = 0, 0
    with open(path, "r", encoding=enc) as f:
        for line in f:
            line = line.strip()
            if not line or not line.startswith("{"):
                continue
            total += 1
            try:
                obj = json.loads(line)
                if isinstance(obj, dict):
                    tweets.append(obj)
            except Exception:
                bad += 1
                continue

    print(f"[load] Registros leídos: {len(tweets)} | líneas con error: {bad}/{total}")
    return tweets

# Ejecutar carga
tweets_raw = load_tweets_from_txt(INPUT_TXT)
len(tweets_raw)


[load] Codificación detectada: utf-16
[load] Registros leídos: 5019 | líneas con error: 0/5019


5019

In [ ]:
def get_username(obj: Dict[str, Any]) -> str:
    u = obj.get("user") or {}
    return (u.get("username") or "").strip() if isinstance(u, dict) else ""

def get_raw_content(obj: Dict[str, Any]) -> str:
    return (obj.get("rawContent") or "").strip()

def get_hashtags(obj: Dict[str, Any]) -> List[str]:
    hs = obj.get("hashtags", [])
    # Pueden venir como lista de strings o lista de objetos
    out = []
    if isinstance(hs, list):
        for h in hs:
            if isinstance(h, str):
                out.append(h.strip())
            elif isinstance(h, dict):
                # por si viniera como {"text": "algo"}
                txt = h.get("text") or h.get("tag") or ""
                if txt:
                    out.append(str(txt).strip())
    return [h for h in out if h]

def get_mentions(obj: Dict[str, Any]) -> List[str]:
    ms = obj.get("mentionedUsers", [])
    out = []
    if isinstance(ms, list):
        for m in ms:
            if isinstance(m, dict):
                u = m.get("username")
                if u:
                    out.append(str(u).strip())
            elif isinstance(m, str):
                out.append(m.strip())
    return [m for m in out if m]

def get_quoted(obj: Dict[str, Any]) -> Dict[str, Any]:
    qt = obj.get("quotedTweet")
    return qt if isinstance(qt, dict) else {}

# Extrae el username del tweet citado (si existe)
def get_from_user_for_row(obj: Dict[str, Any]) -> str:
    qt = get_quoted(obj)
    return get_username(qt) if qt else ""


In [ ]:
def row_from_tweet(obj: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "Username": get_username(obj),
        "rawTweet": get_raw_content(obj),
        "hashtags": ", ".join(get_hashtags(obj)),     # en CSV quedará como texto separado por coma
        "mentions": ", ".join(get_mentions(obj)),     # idem
        "fromUser": get_from_user_for_row(obj),       # usuario del tweet citado por ESTA FILA (si existe)
    }


"""
Agrega una fila por 'obj' y luego recursivamente por cada quotedTweet anidado.
Usa seen_ids para evitar ciclos/duplicados si se repiten objetos por ID.
"""
def flatten_with_quotes(obj: Dict[str, Any], rows: List[Dict[str, Any]], seen_ids: set):
    # Intentar usar 'id' para deduplicación (si existe)
    obj_id = obj.get("id") or obj.get("id_str")
    if obj_id is not None:
        if obj_id in seen_ids:
            return
        seen_ids.add(obj_id)

    # Fila del tweet actual
    rows.append(row_from_tweet(obj))

    # Si tiene quotedTweet, aplanar recursivamente
    qt = get_quoted(obj)
    if qt:
        flatten_with_quotes(qt, rows, seen_ids)

def build_flat_table(objs: List[Dict[str, Any]]) -> pd.DataFrame:
    rows: List[Dict[str, Any]] = []
    seen_ids: set = set()
    for obj in objs:
        if isinstance(obj, dict):
            flatten_with_quotes(obj, rows, seen_ids)
    # Orden de columnas exactamente como pediste
    cols = ["Username", "rawTweet", "hashtags", "mentions", "fromUser"]
    df = pd.DataFrame(rows, columns=cols)
    # Limpieza básica de espacios
    for c in cols:
        df[c] = df[c].fillna("").astype(str).str.strip()
    return df

df_converted = build_flat_table(tweets_raw)
print(df_converted.shape)
df_converted.head(11)


(4944, 5)


,Username,rawTweet,hashtags,mentions,fromUser
0,La_ReVoluZzion,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",,"usembassyguate, 48CantonesToto, USAIDGuate, UE...",XelaNewsGt
1,XelaNewsGt,#URGENTE Lo que los medios #faferos no informa...,"URGENTE, faferos, BernardoArévalo, NebajQuiché...",,
2,M24095273,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,,"IvanDuque, BArevalodeLeon",
3,carlosalbesc,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,,"IvanDuque, BArevalodeLeon",
4,Brenda_AGN,El presidente @BArevalodeLeon y la vicepreside...,,"BArevalodeLeon, KarinHerreraVP, AGN_noticias, ...",
5,Roberto28338166,@BArevalodeLeon El muy hijo de puta inyectó co...,,BArevalodeLeon,
6,LuisEnr36669555,@EmisorasUnidas @BArevalodeLeon Y que de bueno...,,"EmisorasUnidas, BArevalodeLeon",
7,keratox1,@IvanDuque @BArevalodeLeon Productiva es que g...,,"IvanDuque, BArevalodeLeon",
8,Radio_TGW,"@KarinHerreraVP El presidente de la República,...",,"KarinHerreraVP, BArevalodeLeon",
9,Corleone_62,"@mys_servicios @BArevalodeLeon No tampoco, ese...",,"mys_servicios, BArevalodeLeon",


In [9]:
df_converted.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✓ Archivo guardado en: {OUTPUT_CSV}")

✓ Archivo guardado en: ../data/converted_data.csv
